<a href="https://colab.research.google.com/github/severancej/colorful_project/blob/main/Project%20CP1%20-%20%EC%8A%A4%ED%94%84%EB%A6%B0%ED%8A%B8/AI_14_%EC%9E%A5%EC%A7%80%EC%9D%80_CP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

# 🚥CP1 프로젝트 주제 - 서비스이용자를 증가시키기 위한 객체탐지인식 인공지능 모델을 이용한 서비스 이용방법 개선방향 찾기

---
<br><br>

###**💡프로젝트를 진행하는 만드는 이유 -** <br>현재 스프린트는 유료와 무료 서비스로 나뉘어 사용자에게 서비스가 다른게 전달됩니다.같은 식단관리서비스를 제공하는 다른 어플들과의 경쟁력을 확보하고 새로운 사용자의 유입을 높이는 방향으로 프로젝트 계획을 세웠습니다.<br> 식단기록의 편이성을 높이기 위해 물체인식모델인 YOLO(You Only Look Once)v5를 통해 사용자가 오늘 먹은 식단의 사진 한장만 올려도  YOLO(You Only Look Once)v5가 음식을 인식해 자동적으로 검색어로 연결되는 모델을 계획하였습니다.


---



🔔 YOLO 설치하기

In [ ]:
!git clone https://github.com/ultralytics/yolov5

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [ ]:
# 설치된 YOLO 5폴더 안 파일리스트 확인
!ls yolov5

benchmarks.py	 detect.py   models	       setup.cfg       val.py
classify	 export.py   README.md	       train.py
CONTRIBUTING.md  hubconf.py  requirements.txt  tutorial.ipynb
data		 LICENSE     segment	       utils


In [ ]:
# yolo5을 쓰기 위해 requirements.txt로  환경을 맞쳐주기 위해 설치 함
!pip install -r yolov5/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

#glob는 파일들의 리스트를 뽑을 때 사용
from glob import glob

img_list = glob('/content/drive/MyDrive/CP1/dataset/images/*.jpg')

# dataset폴더안에 가지고 있는 이미지가 몇장인지 출력해봄
print(len(img_list))


79


In [ ]:
#훈련세트와 검증세트로 나눠줌

from sklearn.model_selection import train_test_split
#내가 헷갈렸던 부분
#train은 80% val데이터 셋은 20% 나눔
# train_test_split의 파라미터에 test_size지정 했다고
# train 80% 중에 val(검증)세트를 따로 나눠서
# train,val,test 총3개로 나누는 게 아니라
# train_test_split함수는 2가지로만 나눌 수 있다.
train_img_list, val_img_list = train_test_split(img_list, test_size = 0.2 ,random_state = 1000)

#전체 1900장 중에서 나눠 진 train, val의 데이터셋 크기 확인
print(len(train_img_list), len(val_img_list))

63 16


✅전체 데이터 셋 이미지 79장이<br>
- train(훈련) 데이터 셋은 63장<br>
- val(검증) 데이터 셋은 16장으로 나눠짐<br><br>

---

In [ ]:
# train 데이터셋 이미지 경로랑 
# val(검증)데이터셋 이미지 경로를 .txt파일로 저장해줌

with open ('/content/drive/MyDrive/CP1/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n' )

with open ('/content/drive/MyDrive/CP1/dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n' )


#어떤 파일들이 train과 val의 데이터셋이 됐는지 리스트로 있는 것을 
#.txt파일로 만들어 줌
#파일을 열면 해당 이미지들의  경로명이 나열되어 있음

In [ ]:
# data.yaml이  뭘 포함하는지 보려고
# cat(concatenate) : 단순 파일 출력, 파일 여러개를 합치는 기능도 가능
%cat /content/drive/MyDrive/CP1/dataset/data.yaml

path: ../datasets/roboflow

train: train/images
val: valid/images
test: test/images

nc: 19
names: ['- --', '-', '--', '--- - ----', '--- -- -- ---', '--- --', '--- ---', '--- -----', '---', '---- ----', '---- -----', '----', '-----', '------ ----', '------', '-------', '--------', '---------', '----------']

In [ ]:
# data.yaml의 내용을 'r'읽어보고

import yaml

with open('/content/drive/MyDrive/CP1/dataset/data.yaml', 'r') as f:
  data = yaml.safe_load(f)
# data = yaml.load(f)하니깐 에러났는데 safe_load(f)쓰니깐 해결남
# 해결책 참고 https://stackoverflow.com/questions/70981856/how-do-i-fix-the-typeerror-load-missing-1-required-positional-argument-load
print(data)

data['train'] = '/content/drive/MyDrive/CP1/dataset/train.txt'
data['val'] = '/content/drive/MyDrive/CP1/dataset/val.txt'
# 내용수정 하려고
with open('/content/drive/MyDrive/CP1/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'path': '../datasets/roboflow', 'train': 'train/images', 'val': 'valid/images', 'test': 'test/images', 'nc': 19, 'names': ['- --', '-', '--', '--- - ----', '--- -- -- ---', '--- --', '--- ---', '--- -----', '---', '---- ----', '---- -----', '----', '-----', '------ ----', '------', '-------', '--------', '---------', '----------']}
{'path': '../datasets/roboflow', 'train': '/content/drive/MyDrive/CP1/dataset/train.txt', 'val': '/content/drive/MyDrive/CP1/dataset/val.txt', 'test': 'test/images', 'nc': 19, 'names': ['- --', '-', '--', '--- - ----', '--- -- -- ---', '--- --', '--- ---', '--- -----', '---', '---- ----', '---- -----', '----', '-----', '------ ----', '------', '-------', '--------', '---------', '----------']}


In [ ]:
#현재 위치 확인
%pwd

'/content'

In [ ]:
# yolo5 설치한곳으로 이동
%cd /content/yolov5/

# 모델을 훈련시키기 위해
# 이미지 크기는 416으로 하고
# chg는 모델의 구조를 나타내는데 위치는 
# /content/yolov5/models 에
# yolo5l.yaml, yolo5m.yaml, yolo5n.yaml, yolo5s.yaml, yolo5x.yaml 
# 파일들을 말한다.

# 그중에 /content/yolov5/models/yolov5s.yaml모델을 써본다다

!python train.py --img 416 --batch 16 --epochs 50 --data /content/drive/MyDrive/CP1/dataset/data.yaml --cfg /content/yolov5/models/yolov5s.yaml --weights yolov5s.pt --name Traffic_Light_yolov5s_results
# --weights yolov5s.pt 는 가중치는 pre-train 된 모델을 다운로드 받아서 사용해라는 뜻
# --name Traffic_Light_yolov5s_results 실행결과 나온 결과의 이름을 지정함함

/content/yolov5
train: weights=yolov5s.pt, cfg=/content/yolov5/models/yolov5s.yaml, data=/content/drive/MyDrive/CP1/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=Traffic_Light_yolov5s_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-239-gf33718f Python-3.7.15 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.

In [ ]:
# 내가 하고 있는게 맞는지.. 왜 트레이닝은 안될까요.. 레이블폴더도 있는데...
# roboflow로 라벨링을 한글로 하니깐 ___이런식으로 됨.